In this notebook we load our best models.
The checkpoints and best results can be found at https://drive.google.com/drive/folders/1-1_UPvtHjzOAblCphf6iLebpFqaEi_hl.

In [ ]:
# This cell is for Colab
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir an2dl_1
%cd an2dl_1
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c artificial-neural-networks-and-deep-learning-2020
!unzip artificial-neural-networks-and-deep-learning-2020.zip

In [2]:
# This cell is for Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import json

SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input 

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=15,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        validation_split=0.2,
                                        preprocessing_function=preprocess_input)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = './MaskDataset'

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256
num_classes=3

with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)
dataframe = pd.DataFrame(dic.items())
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)
dataframe = dataframe.sample(frac=1).reset_index(drop=True)

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')  # targets are directly converted into one-hot vectors


# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs, 
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED,
                                               subset='validation')

Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [5]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Shuffle (Already done in generator..)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

Now the best models will be created and loaded

In [6]:
en = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [7]:
model1 = tf.keras.Sequential()
model1.add(en)
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(units=512, activation='relu'))
model1.add(tf.keras.layers.Dense(units=256, activation='relu'))
model1.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------
model1.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [8]:
inc = tf.keras.applications.InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(img_h,img_w,3))

In [9]:
model2 = tf.keras.Sequential()
model2.add(inc)
model2.add(tf.keras.layers.GlobalMaxPooling2D())
model2.add(tf.keras.layers.Dense(units=512, activation='relu'))
model2.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------
model2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
xce = tf.keras.applications.Xception(weights='imagenet',include_top=False,input_shape=(img_h,img_w,3))

In [11]:
model3 = tf.keras.Sequential()
model3.add(xce)
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(units=512, activation='relu'))
model3.add(tf.keras.layers.Dense(units=256, activation='relu'))
model3.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------
model3.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [12]:
enb6 = tf.keras.applications.EfficientNetB6(weights='imagenet',include_top=False,input_shape=(img_h,img_w,3))

In [13]:
model4 = tf.keras.Sequential()
model4.add(enb6)
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(units=512, activation='relu'))
model4.add(tf.keras.layers.Dense(units=256, activation='relu'))
model4.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------
model4.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
model1.load_weights("Classification_experiments/CNN_EffB7/best.ckpt")

In [15]:
model2.load_weights("Classification_experiments/CNN_InceptionResNet/best.ckpt")

In [16]:
model3.load_weights("Classification_experiments/CNN_Xception/best.ckpt")

In [17]:
model4.load_weights("Classification_experiments/CNN_EffB6/best.ckpt")

In [18]:
# Test
import os
from datetime import datetime
exps_dir = 'Classification_experiments/Final'
res_dir = os.path.join(exps_dir, 'Results')
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
def create_csv(results, results_dir=res_dir):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                              test_dir,
                                              batch_size=bs,
                                              target_size=(img_h, img_w),
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed=SEED)


test_gen.reset()

predictions1 = model1.predict_generator(test_gen, len(test_gen), verbose=1)
predictions2 = model2.predict_generator(test_gen, len(test_gen), verbose=1)
predictions3 = model3.predict_generator(test_gen, len(test_gen), verbose=1)
predictions4 = model4.predict_generator(test_gen, len(test_gen), verbose=1)

pred_avg = (predictions1 + predictions2 + predictions3 + predictions4) / 4

results = {}
images = test_gen.filenames
i = 0

for p in pred_avg:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results)

Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
29/29 [==============================] - 7s 241ms/step
